In [70]:
#..........................案例1:
#.......................... FP-growth算法
#..........第一步：计算单集，根据Apriori原理和阈值进行删减。
#..........第二步：计算单集，根据Apriori和阈值进行删减。
# 构件树
# 数据结构
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {}

    def inc(self, numOccur):
        self.count += numOccur

    def disp(self, ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)
node=treeNode('pyramid',9,None)
node.children['eye']=treeNode('eye',13,None)
print('数据结构：')
node.disp()
# 树
# 辅助函数
def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)
# 主函数
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet: #first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable.keys()):  #移除不满足min的单项
        if headerTable[k] < minSup:
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  # 根据全局频率排序
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count) #排序后填充树
    return retTree, headerTable #return tree and header table
simpDat = [['r', 'z', 'h', 'j', 'p'],['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],['r', 'x', 'n', 'o', 's'],['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
initDat={}
for trans in simpDat:
    initDat[frozenset(trans)] = 1
Tree,Header=createTree(initDat,3)
print('单维：\n',[i for i in Header])
print('树：\n')
Tree.disp()
# 遍历树,构建条件模式基
# 如x:{z}3,{}1; y:{z,x}3
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats
print('条件模式基：\n',findPrefixPath('x',Header['x'][1]))
# 寻找频繁集
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: str(p[1]))] #(sort header table)
    for basePat in bigL:  #start from bottom of header table
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        #print 'finalFrequent Item: ',newFreqSet    #append to set
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        #print 'condPattBases :',basePat, condPattBases
        #2. construct cond FP-tree from cond. pattern base
        myCondTree, myHead = createTree(condPattBases, minSup)
        #print 'head from conditional tree: ', myHead
        if myHead != None: #3. mine cond. FP-tree
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)
freq=[]
mineTree('myFP',Header,3,set([]),freq)
print('频繁集：')
freq

数据结构：
   pyramid   9
     eye   13
单维：
 ['z', 'r', 't', 's', 'y', 'x']
树：

   Null Set   1
     z   5
       r   1
       x   3
         t   3
           s   1
             y   1
           y   2
             r   1
             s   1
     x   1
       s   1
         r   1
条件模式基：
 {frozenset({'z'}): 3}
频繁集：


[{'y'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'t', 'x', 'y', 'z'},
 {'t', 'x', 'y'},
 {'t', 'y'},
 {'y', 'z'},
 {'t', 'y', 'z'},
 {'r'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'x'},
 {'t', 'x', 'z'},
 {'t', 'z'},
 {'x'},
 {'x', 'z'},
 {'z'}]